In [19]:
import torch
import torch.nn as nn

In [ ]:
import wandb
!wandb login

In [21]:
from dataclasses import dataclass


@dataclass 
class ModelArgs:
    device = 'cuda'
    no_of_neurons = 16
    block_size = 16
    batch_size = 16
    dropout = 0.1
    epoch = 50
    max_lr = 1e-4
    embedding_dims: int = 784

In [22]:
torch.set_default_device(ModelArgs.device)

In [23]:
import torch
from torch.utils.data import Dataset, DataLoader


torch.manual_seed(0)


num_samples = 10000 
seq_length = ModelArgs.block_size  
device = ModelArgs.device  


t = torch.linspace(0, 100, num_samples + seq_length, device=device)
data = torch.sin(t) 
# data = t

X_tensor = torch.stack([data[i:i+seq_length] for i in range(num_samples)])
y_tensor = data[seq_length:]  # Next value prediction

train_size = int(0.8 * num_samples)

X_train, y_train = X_tensor[:train_size], y_tensor[:train_size]  
X_val, y_val = X_tensor[train_size:], y_tensor[train_size:]  


class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


train_dataset = TimeSeriesDataset(X_train, y_train)
val_dataset = TimeSeriesDataset(X_val, y_val)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


generator = torch.Generator(device=device)


train_loader = DataLoader(
    train_dataset,
    batch_size=ModelArgs.batch_size,
    shuffle=True,  
    generator=generator,  
    # drop_last=True
)

val_loader = DataLoader(
    val_dataset,
    generator=generator, 
    # drop_last=True,
    batch_size=ModelArgs.batch_size,
    shuffle=True, 
)



In [26]:
class ResetGate(torch.nn.Module):
    def __init__(self, device, no_of_neurons):
        super().__init__()
        self.linear = nn.Linear(in_features=ModelArgs.no_of_neurons + 1, out_features=no_of_neurons, device=device, dtype=torch.float32)
   
    def forward(self, x, ht_1):
        # print("Reset: ", x.shape)
        # print("Reset: ", ht_1.shape)
        x = torch.cat([x, ht_1], dim=-1)
        out = torch.nn.functional.sigmoid(self.linear(x))
        return out

In [27]:
class UpdateGate(nn.Module):
    def __init__(self, device, no_of_neurons) -> None:
        super().__init__()
        self.linear = nn.Linear(in_features=ModelArgs.no_of_neurons + 1, out_features=no_of_neurons, device=device, dtype=torch.float32)
    def forward(self, x, ht_1):
        x = torch.cat([x, ht_1], dim=-1)
        out = torch.nn.functional.sigmoid(self.linear(x))
        return out

In [29]:

class GRUBlock(nn.Module):
    def __init__(self, device, no_of_neurons) -> None:
        super().__init__()
        self.update = UpdateGate(device=device, no_of_neurons=no_of_neurons)
        self.reset = ResetGate(device=device, no_of_neurons=no_of_neurons)
        self.candidate = nn.Linear(in_features=ModelArgs.no_of_neurons + 1, out_features=no_of_neurons, device=device, dtype=torch.float32)
        self.linear_layer = nn.Linear(in_features=ModelArgs.no_of_neurons, out_features=1, device=device)
        self.no_of_neurons = no_of_neurons
    def forward(self, x, output=None):
        
        ht_1 = nn.Parameter(torch.zeros(x.shape[0], self.no_of_neurons, device=device, requires_grad=True, dtype=torch.float32)) #we do not want to cary info between sequences or layers
        
        # ht = ht_1
        # print(x.shape)
        seq_len = x.shape[1]
        
        # print("LSTM: ",x.shape)
        # print("LSTM: ", self.ht_1.shape)
        # print("LSTM: ", self.ct_1.shape)
        # ht = None
        # combined = None
        if(output is None):
            output = []
            for t in range(seq_len):
                
                xt = x[:, t].unsqueeze(-1)
                
                # print(x.shape)
                # print(xt.shape)
                # print(ht_1.shape)
                reset_out = self.reset(xt, ht_1) #It is not updating the states correctly since using the same hidden states
                # print(xt)
                ht_1_candidate = nn.functional.tanh(self.candidate(torch.cat([reset_out * ht_1, xt], dim=-1)))
                update_out = self.update(xt, ht_1)
                ht = ((1- update_out) * ht_1) + (update_out * ht_1_candidate)
                ht_1 = ht
                output.append(ht_1)
        
            return ht_1, torch.stack(output, dim=1)
        
            # print("Current hidden state: ", ht)
        elif(output is not None and len(output) != 0):
            new_output = []
            for t in range(seq_len):
                
                # ht = x[:, t].unsqueeze(-1)
                xt = output[:, t]
                xt = self.linear_layer(xt)
                # print(x.shape)
                # print("Now: ", xt.shape)
                # print(ht_1.shape)
                reset_out = self.reset(xt, ht_1) #It is not updating the states correctly since using the same hidden states
                # print(xt)
                ht_1_candidate = nn.functional.tanh(self.candidate(torch.cat([reset_out * ht_1, xt], dim=-1)))
                update_out = self.update(xt, ht_1)
                ht = ((1- update_out) * ht_1) + (update_out * ht_1_candidate)
                ht_1 = ht
                new_output.append(ht_1)
                # print("Another block: ", output[:, t])
        
            return ht_1, torch.stack(new_output, dim=1)

In [30]:
class GRU(nn.Module):
    def __init__(self, device, no_of_neurons, out_features):
        super().__init__()
        self.block1 = GRUBlock(device=device, no_of_neurons=no_of_neurons)
        # self.block2 = GRUBlock(device=device, no_of_neurons=no_of_neurons)
        # self.block3 = GRUBlock(device=device, no_of_neurons=no_of_neurons)
        # self.ht_1 = torch.randn(ModelArgs.batch_size, no_of_neurons, device=device, requires_grad=True, dtype=torch.float32)
        # self.ct_1 = torch.randn(ModelArgs.batch_size, no_of_neurons,device=device, requires_grad=True, dtype=torch.float32)
        self.output = nn.Linear(in_features=ModelArgs.no_of_neurons, out_features=out_features, device=device, dtype=torch.float32)
        self.dropout = nn.Dropout(p=ModelArgs.dropout)
        # self.embedding = nn.Embedding()
        
    def forward(self, x):
        # x = 
        # print("LSTM: ",x.shape)
        # print("LSTM: ", self.ht_1.shape)
        # print("LSTM: ", self.ct_1.shape)
        ht, hidden_states = self.block1(x)
        # print("Ouputs: ", hidden_states)
        # print(ht.shape)
        # print(ct.shape)
        # ht, hidden_states = self.block2(x, hidden_states)
        # ht, hidden_states = self.block3(x, hidden_states)
        ht = self.dropout(ht)
        # print("After: ", ht.shape)
        out = self.output(ht)
        return out

In [31]:
class DeepGRU(nn.Module):
    def __init__(self, device, no_of_neurons, out_features):
        super().__init__()
        self.block1 = GRUBlock(device=device, no_of_neurons=no_of_neurons)
        self.block2 = GRUBlock(device=device, no_of_neurons=no_of_neurons)
        self.block3 = GRUBlock(device=device, no_of_neurons=no_of_neurons)
        # self.ht_1 = torch.randn(ModelArgs.batch_size, no_of_neurons, device=device, requires_grad=True, dtype=torch.float32)
        # self.ct_1 = torch.randn(ModelArgs.batch_size, no_of_neurons,device=device, requires_grad=True, dtype=torch.float32)
        self.output = nn.Linear(in_features=ModelArgs.no_of_neurons, out_features=out_features, device=device, dtype=torch.float32)
        self.dropout = nn.Dropout(p=ModelArgs.dropout)
        # self.embedding = nn.Embedding()
        
    def forward(self, x):
        # x = 
        # print("LSTM: ",x.shape)
        # print("LSTM: ", self.ht_1.shape)
        # print("LSTM: ", self.ct_1.shape)
        ht, hidden_states = self.block1(x)
        # print("Ouputs: ", hidden_states)
        # print(ht.shape)
        # print(ct.shape)
        ht, hidden_states = self.block2(x, hidden_states)
        ht, hidden_states = self.block3(x, hidden_states)
        ht = self.dropout(ht)
        # print("After: ", ht.shape)
        out = self.output(ht)
        return out

In [32]:
# model = GRU(device=ModelArgs.device, no_of_neurons=ModelArgs.no_of_neurons, out_features=1)
model = DeepGRU(device=ModelArgs.device, no_of_neurons=ModelArgs.no_of_neurons, out_features=1)
model = model.to(ModelArgs.device)

In [ ]:
!pip install torchinfo

from torchinfo import summary

x = torch.randint(0, 100, (ModelArgs.batch_size,ModelArgs.block_size))  # Random integer between 0 and 100
x = x.to(ModelArgs.device)
summary(model=model,
        input_data=x,
        # input_size=(ModelArgs.batch_size, ModelArgs.block_size, ModelArgs.embeddings_dims),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])


In [35]:
criterion = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=ModelArgs.max_lr)

In [ ]:
model.train()
train_losses =  torch.zeros(len(train_loader))
val_losses = torch.zeros(len(val_loader))
wandb.init(
    project='GRU-From-Scratch'
)
for epoch in range(ModelArgs.epoch):
    
    count = 0
    for X, y in train_loader:
        y_pred = model(X)
        # print(y_pred.shape)
        loss = criterion(y_pred, y)
        train_losses[count] = loss.item()
        # print("Loss: ", loss.item())
        
        # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        # if(count != 0):
        #     total_norm_before = torch.norm(
        #             torch.stack([torch.norm(p.grad.detach(), 2) for p in model.parameters()]), 2
        #         )

        #     torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        #         # Compute gradient norms after clipping
        #     total_norm_after = torch.norm(
        #             torch.stack([torch.norm(p.grad.detach(), 2) for p in model.parameters()]), 2
        #         )
                
        #     # if(device  == 0 and step !=0):
        #     print(f"Gradient Norm Before Clipping: {total_norm_before.item():.4f}")
        #     print(f"Gradient Norm After Clipping: {total_norm_after.item():.4f}")
        
        optimizer.zero_grad(set_to_none=True)
        loss.backward(retain_graph=True)
        optimizer.step()
        count += 1
        
    #     wandb.log({
    #   "Train Loss": loss.item(),
    #   "Val Loss": loss.item(),
    #   "step": count  
    # })
    # count = 0
    print("Epoch: ", epoch, "|", "Step: ", count, "|", "Train Loss: ", loss.item())
    model.eval()
    count = 0
    
    for X, y in val_loader:
        y_pred = model(X)
        # print(y_pred.shape)
        loss = criterion(y_pred, y)
        
        # print("Loss: ", loss.item())
        val_losses[count] = loss.item()
        
        # optimizer.zero_grad()
        # loss.backward()
        # optimizer.step()
        count += 1
    #     wandb.log({
    #   "Train Loss": loss.item(),
    #   "Val Loss": loss.item(),
    #   "step": count  
    # })
        print("Epoch: ", epoch, "|", "Step: ", count, "|", "Val Loss: ", loss.item())
    model.train()
    wandb.log({
      "Train Loss": train_losses.mean(),
      "Val Loss": val_losses.mean(),
      "epoch": epoch  
    })
    print("Epoch: ", epoch, "|", "Train Loss: ", train_losses.mean(),  "|", "Val Loss: ", val_losses.mean())
